In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
import pandas as pd
#path = f"/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/DFallpa_Diego_230929_RULE_BASE_stroke.pkl"
animal = "Diego"
date = "230630"
path= f"/home/kgg/Desktop/eyetracking_analyses/{animal}-{date}/dfallpa.pkl"
DFallpa = pd.read_pickle(path)

In [4]:
DFallpa

,which_level,event,bregion,twind,pa,pa_x_shape
0,flex,fixon_preparation,M1_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(23, 2381, 80)"
1,flex,fixon_preparation,M1_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(18, 2381, 80)"
2,flex,fixon_preparation,PMv_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(43, 2381, 80)"
3,flex,fixon_preparation,PMv_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(36, 2381, 80)"
4,flex,fixon_preparation,PMd_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(24, 2381, 80)"
5,flex,fixon_preparation,PMd_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(19, 2381, 80)"
6,flex,fixon_preparation,dlPFC_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(27, 2381, 80)"
7,flex,fixon_preparation,vlPFC_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(32, 2381, 80)"
8,flex,fixon_preparation,vlPFC_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(18, 2381, 80)"
9,flex,fixon_preparation,FP_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(17, 2381, 80)"


### Do normalization of activity, if not already done

In [ ]:
# from neuralmonkey.classes.population_mult import po
path_to_save_example_fr_normalization = None
fr_normalization_method = "each_time_bin"

if fr_normalization_method is not None:
    if fr_normalization_method=="each_time_bin":
        # Then demean in each time bin indepednently
        subtract_mean_at_each_timepoint = True
        subtract_mean_across_time_and_trial = False
    elif fr_normalization_method=="across_time_bins":
        # ALl time bins subtract the same scalar --> maintains temporal moudlation.
        subtract_mean_at_each_timepoint = False
        subtract_mean_across_time_and_trial = True
    else:
        print(fr_normalization_method)
        assert False

    from neuralmonkey.analyses.state_space_good import popanal_preprocess_scalar_normalization
    list_panorm = []

    for i, pa in enumerate(DFallpa["pa"].tolist()):
        if path_to_save_example_fr_normalization is not None and i==0:
            plot_example_chan_number = pa.Chans[0]
            if which_level=="trial":
                plot_example_split_var_string = "seqc_0_shape"
            elif which_level=="stroke":
                plot_example_split_var_string = "shape"
            else:
                plot_example_split_var_string = q_params["effect_vars"][0]
        else:
            plot_example_chan_number = None
            plot_example_split_var_string = None
        PAnorm, PAscal, PAscalagg, fig, axes, groupdict = popanal_preprocess_scalar_normalization(pa, None,
                                                                                          DO_AGG_TRIALS=False,
                                                                                          plot_example_chan_number=plot_example_chan_number,
                                                                                            plot_example_split_var_string = plot_example_split_var_string,
                                                                                          subtract_mean_at_each_timepoint=subtract_mean_at_each_timepoint,
                                                                                          subtract_mean_across_time_and_trial=subtract_mean_across_time_and_trial)
        if path_to_save_example_fr_normalization is not None and i==0:
            savefig(fig, path_to_save_example_fr_normalization)
        list_panorm.append(PAnorm)
    DFallpa["pa"] = list_panorm

# Decoders

In [5]:
## Cross-temporal decoder, trained on seqc_0_shape, tested on shape-fixation

from neuralmonkey.analyses.decode_good import decodewrapouterloop_categorical_cross_time_cross_var, decodewrapouterloop_categorical_cross_time_plot

SAVEDIR_ANALYSIS = f"/home/kgg/Desktop/eyetracking_analyses/DECODERS/{animal}-{date}"
SAVEDIR = f"{SAVEDIR_ANALYSIS}/cross_temporal_diff_var_train_test"
os.makedirs(SAVEDIR, exist_ok=True)

time_bin_size = 0.1
slide=0.05

list_var_decode_train_test = [('seqc_0_shape', 'shape-fixation'), ('seqc_0_shape', 'seqc_0_shape'), ('shape-fixation', 'shape-fixation')]

assert len(DFallpa["twind"].unique())==1, "not big deal. just change code below to iter over all (ev, tw)."

DFRES = decodewrapouterloop_categorical_cross_time_cross_var(DFallpa,
                                                     list_var_decode_train_test,
                                                         time_bin_size, slide, savedir=SAVEDIR, extract_params=False,
                                                            ignore_same_events=False, which_level="flex")
if len(DFRES)>0:
    decodewrapouterloop_categorical_cross_time_plot(DFRES, SAVEDIR)

M1_m (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     83
arcdeep-4-3-0 :     111
line-8-1-0 :     82
zigzagSq-1-1-1 :     105
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=428
level line-8-1-0, keeping, n=376
level arcdeep-4-3-0, keeping, n=600
level zigzagSq-1-1-1, keeping, n=596
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 428', 'arcdeep-4-3-0 : 600', 'line-8-1-0 : 376', 'zigzagSq-1-1-1 : 596']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     428
arcdeep-4-3-0 :     600
line-8-1-0 :     376
zigzagSq-1-1-1 :     596
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved t

--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/4_cross_temporal_diff_var_train_test/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     83
arcdeep-4-3-0 :     111
line-8-1-0 :     82
zigzagSq-1-1-1 :     105
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved to:  /ho

--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=491
level zigzagSq-1-1-1, keeping, n=547
level line-8-1-0, keeping, n=380
level arcdeep-4-3-0, keeping, n=582
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 491', 'arcdeep-4-3-0 : 582', 'line-8-1-0 : 380', 'zigzagSq-1-1-1 : 547']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/4_cross_temporal_diff_var_train_test/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     491
arcdeep-4-3-0 :     582
line-8-1-0 :     380
zigzagSq-1-1-1 :     547
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']

--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/4_cross_temporal_diff_var_train_test/PMv_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     83
arcdeep-4-3-0 :     111
line-8-1-0 :     82
zigzagSq-1-1-1 :     105
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved

...

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=428
level line-8-1-0, keeping, n=376
level arcdeep-4-3-0, keeping, n=600
level zigzagSq-1-1-1, keeping, n=596
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=491
level zigzagSq-1-1-1, keeping, n=547
level line-8-1-0, keeping, n=380
level arcdeep-4-3-0, keeping, n=582
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 428', 'arcdeep-4-3-0 : 600', 'line-8-1-0 : 376', 'zigzagSq-1-1-1 : 596']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/4_cross_temporal_diff_var_train_test/SMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     428
arcdeep-4-3-0 :     600
line-8-1-0 :     376
zigzagSq-1-1-1 :     596
['var_decode', 'V-2-2-0 : 491', 'arcdeep-4-3-0 : 582', 'line-8-1-0 : 380', 'zigzagSq-1-1-1 : 547

--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=491
level zigzagSq-1-1-1, keeping, n=547
level line-8-1-0, keeping, n=380
level arcdeep-4-3-0, keeping, n=582
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/4_cross_temporal_diff_var_train_test/preSMA_p-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     83
arcdeep-4-3-0 :     111
line-8-1-0 :     82
zigzagSq-1-1-1 :     105
['var_decode', 'V-2-2-0 : 491', 'arcdeep-4-3-0 : 582', 'line-8-1-0 : 380', 'zigzagSq-1-1-1 : 547']
S

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=428
level line-8-1-0, keeping, n=376
level arcdeep-4-3-0, keeping, n=600
level zigzagSq-1-1-1, keeping, n=596
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=428
level line-8-1-0, keeping, n=376
level arcdeep-4-3-0, keeping, n=600
level zigzagSq-1-1-1, keeping, n=596
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 428', 'arcdeep-4-3-0 : 600', 'line-8-1-0 : 376', 'zigzagSq-1-1-1 : 596']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/4_cross_temporal_diff_var_train_test/preSMA_p-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     428
arcdeep-4-3-0 :     600
line-8-1-0 :     376
zigzagSq-1-1-1 :     596
['var_decode', 'V-2-2-0 : 428', 'arcdeep-4-3-0 : 600', 'line-8-1-0 : 376', 'zigzagSq-1-1-1 : 596'

--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=428
level line-8-1-0, keeping, n=376
level arcdeep-4-3-0, keeping, n=600
level zigzagSq-1-1-1, keeping, n=596
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     83
arcdeep-4-3-0 :     111
line-8-1-0 :     82
zigzagSq-1-1-1 :     105
['var_decode', 'V-2-2-0 : 428', 'arcdeep-4-3-0 : 600', 'line-8-1-0 : 376', 'zigzagSq-1-1-1 : 596']
Saved

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=428
level line-8-1-0, keeping, n=376
level arcdeep-4-3-0, keeping, n=600
level zigzagSq-1-1-1, keeping, n=596
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=428
level line-8-1-0, keeping, n=376
level arcdeep-4-3-0, keeping, n=600
level zigzagSq-1-1-1, keeping, n=596
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 428', 'arcdeep-4-3-0 : 600', 'line-8-1-0 : 376', 'zigzagSq-1-1-1 : 596']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     428
arcdeep-4-3-0 :     600
line-8-1-0 :     376
zigzagSq-1-1-1 :     596
['var_decode', 'V-2-2-0 : 428', 'arcdeep-4-3-0 : 600', 'line-8-1-0 : 376', 'zigzagSq-1-1-1 : 596'

In [ ]:
# result: looks like you can decode in PMv_m, PMv_l pretty well.
# questions:
# - does effect differ for first macrosaccade vs. the rest? (e.g. no pre-fixation effect if macrosacc=1)
# - after subtracting mean (to approximately account for confound), can we still decode shapefix well?

In [ ]:
# next, do the same decoder, but this time adjust for the mean..

In [6]:
# make a new pa with twice the columns: for each brain region, two "fake" events (train and test)
# each pa should have the mean subtracted.
DFallpa_mean_adj = DFallpa.copy()
DFallpa_mean_adj = pd.DataFrame(np.repeat(DFallpa_mean_adj.values, 2, axis=0))
DFallpa_mean_adj.columns = DFallpa.columns

CONJ_VARS_1 = ("shape-macrosaccade-index", "shape-fixation", "loc-fixation", "seqc_0_loc")
CONJ_VARS_2 = ("shape-macrosaccade-index", "seqc_0_shape", "seqc_0_loc", "loc-fixation")

nr = DFallpa_mean_adj.shape[0]
for i in range(0,nr):
    if i%2==0:
        event = "fixon_preparation_train"
        CONJ_VARS = CONJ_VARS_1
    else:
        event = "fixon_preparation_test"
        CONJ_VARS = CONJ_VARS_2
    pa_old = DFallpa_mean_adj.iloc[i]['pa']
    pa_new = pa_old.norm_by_label_subtract_mean("trials", CONJ_VARS)
    DFallpa_mean_adj.iloc[i]['pa'] = pa_new
    DFallpa_mean_adj.iloc[i]['event'] = event

In [7]:
DFallpa_mean_adj

,which_level,event,bregion,twind,pa,pa_x_shape
0,flex,fixon_preparation_train,M1_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(23, 2381, 80)"
1,flex,fixon_preparation_test,M1_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(23, 2381, 80)"
2,flex,fixon_preparation_train,M1_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(18, 2381, 80)"
3,flex,fixon_preparation_test,M1_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(18, 2381, 80)"
4,flex,fixon_preparation_train,PMv_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(43, 2381, 80)"
5,flex,fixon_preparation_test,PMv_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(43, 2381, 80)"
6,flex,fixon_preparation_train,PMv_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(36, 2381, 80)"
7,flex,fixon_preparation_test,PMv_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(36, 2381, 80)"
8,flex,fixon_preparation_train,PMd_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(24, 2381, 80)"
9,flex,fixon_preparation_test,PMd_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(24, 2381, 80)"


In [8]:
## Cross-temporal decoder, trained on seqc_0_shape, tested on shape-fixation

from neuralmonkey.analyses.decode_good import decodewrapouterloop_categorical_cross_time_cross_var, decodewrapouterloop_categorical_cross_time_plot

SAVEDIR_ANALYSIS = f"/home/kgg/Desktop/eyetracking_analyses/DECODERS/{animal}-{date}"
SAVEDIR = f"{SAVEDIR_ANALYSIS}/cross_temporal_diff_var_train_test_MEAN_ADJUSTED"
os.makedirs(SAVEDIR, exist_ok=True)

time_bin_size = 0.1
slide=0.05

list_var_decode_train_test = [('seqc_0_shape', 'shape-fixation'), ('seqc_0_shape', 'seqc_0_shape'), ('shape-fixation', 'shape-fixation')]

DFRES = decodewrapouterloop_categorical_cross_time_cross_var(DFallpa_mean_adj,
                                                     list_var_decode_train_test,
                                                         time_bin_size, slide, savedir=SAVEDIR, extract_params=False,
                                                            ignore_same_events=False, which_level="flex")
if len(DFRES)>0:
    decodewrapouterloop_categorical_cross_time_plot(DFRES, SAVEDIR)

M1_m (-0.4, 0.4) fixon_preparation_train fixon_preparation_train
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_MEAN_ADJUSTED/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_train-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     83
arcdeep-4-3-0 :     111
line-8-1-0 :     82
zigzagSq-1-1-1 :     105
['var_decode', 'V-

--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=596
level line-8-1-0, keeping, n=376
level V-2-2-0, keeping, n=428
level arcdeep-4-3-0, keeping, n=600
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 428', 'arcdeep-4-3-0 : 600', 'line-8-1-0 : 376', 'zigzagSq-1-1-1 : 596']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_MEAN_ADJUSTED/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     428
arcdeep-4-3-0 :     600
line-8-1-0 :     376
zigzagSq-1-1-1 :     596
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zi

M1_m (-0.4, 0.4) fixon_preparation_train fixon_preparation_test
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_MEAN_ADJUSTED/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     83
arcdeep-4-3-0 :     111
line-8-1-0 :     82
zigzagSq-1-1-1 :     105
['var_decode', 'V-2-

--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=596
level line-8-1-0, keeping, n=376
level V-2-2-0, keeping, n=428
level arcdeep-4-3-0, keeping, n=600
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 428', 'arcdeep-4-3-0 : 600', 'line-8-1-0 : 376', 'zigzagSq-1-1-1 : 596']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_MEAN_ADJUSTED/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     428
arcdeep-4-3-0 :     600
line-8-1-0 :     376
zigzagSq-1-1-1 :     596
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zig

M1_m (-0.4, 0.4) fixon_preparation_test fixon_preparation_train
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_MEAN_ADJUSTED/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     83
arcdeep-4-3-0 :     111
line-8-1-0 :     82
zigzagSq-1-1-1 :     105
['var_decode', 'V-2-

...

preSMA_a (-0.4, 0.4) fixon_preparation_test fixon_preparation_train
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_MEAN_ADJUSTED/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     83
arcdeep-4-3-0 :     111
line-8-1-0 :     82
zigzagSq-1-1-1 :     105
['var_decode

--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=596
level line-8-1-0, keeping, n=376
level V-2-2-0, keeping, n=428
level arcdeep-4-3-0, keeping, n=600
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 428', 'arcdeep-4-3-0 : 600', 'line-8-1-0 : 376', 'zigzagSq-1-1-1 : 596']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_MEAN_ADJUSTED/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     428
arcdeep-4-3-0 :     600
line-8-1-0 :     376
zigzagSq-1-1-1 :     596
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 

preSMA_a (-0.4, 0.4) fixon_preparation_test fixon_preparation_test
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', 'zigzagSq-1-1-1 : 105']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_MEAN_ADJUSTED/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     83
arcdeep-4-3-0 :     111
line-8-1-0 :     82
zigzagSq-1-1-1 :     105
['var_decode',

--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=596
level line-8-1-0, keeping, n=376
level V-2-2-0, keeping, n=428
level arcdeep-4-3-0, keeping, n=600
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=105
level line-8-1-0, keeping, n=82
level V-2-2-0, keeping, n=83
level arcdeep-4-3-0, keeping, n=111
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 428', 'arcdeep-4-3-0 : 600', 'line-8-1-0 : 376', 'zigzagSq-1-1-1 : 596']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_MEAN_ADJUSTED/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     428
arcdeep-4-3-0 :     600
line-8-1-0 :     376
zigzagSq-1-1-1 :     596
['var_decode', 'V-2-2-0 : 83', 'arcdeep-4-3-0 : 111', 'line-8-1-0 : 82', '

324000
Plotting bregion:  M1_m
Plotting bregion:  M1_l
Plotting bregion:  PMv_l
Plotting bregion:  PMv_m
Plotting bregion:  PMd_p
Plotting bregion:  PMd_a
Plotting bregion:  dlPFC_a
Plotting bregion:  vlPFC_p
Plotting bregion:  vlPFC_a
Plotting bregion:  FP_p
Plotting bregion:  FP_a
Plotting bregion:  SMA_p
Plotting bregion:  SMA_a
Plotting bregion:  preSMA_p
Plotting bregion:  preSMA_a
=== PLOTTING (all the following plots): prims_single prims_single fixon_preparation_train fixon_preparation_train
Plotting var_decode_train, var_decode_test:  seqc_0_shape seqc_0_shape
Plotting var_decode_train, var_decode_test:  seqc_0_shape shape-fixation
Plotting var_decode_train, var_decode_test:  shape-fixation seqc_0_shape
! no data...
Plotting var_decode_train, var_decode_test:  shape-fixation shape-fixation
=== PLOTTING (all the following plots): prims_single prims_single fixon_preparation_train fixon_preparation_test
Plotting var_decode_train, var_decode_test:  seqc_0_shape seqc_0_shape
Plottin

In [9]:
# first macrosaccade
DFallpa_ifmt = DFallpa.copy()

nr = DFallpa_ifmt.shape[0]
for i in range(0,nr):
    pa_old = DFallpa_ifmt.iloc[i]['pa']
    pa_new = pa_old.slice_by_labels('trials', 'shape-macrosaccade-index', [1])
    DFallpa_ifmt.iloc[i]['pa'] = pa_new

SAVEDIR_ANALYSIS = f"/home/kgg/Desktop/eyetracking_analyses/DECODERS/{animal}-{date}"
SAVEDIR = f"{SAVEDIR_ANALYSIS}/cross_temporal_diff_var_train_test_FIRST_MACROSACCADE"
os.makedirs(SAVEDIR, exist_ok=True)

DFRES = decodewrapouterloop_categorical_cross_time_cross_var(DFallpa_ifmt,
                                                     list_var_decode_train_test,
                                                         time_bin_size, slide, savedir=SAVEDIR, extract_params=False,
                                                            ignore_same_events=False, which_level="flex")
if len(DFRES)>0:
    decodewrapouterloop_categorical_cross_time_plot(DFRES, SAVEDIR)

M1_m (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3-0 : 43', 'line-8-1-0 : 42', 'zigzagSq-1-1-1 : 46']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_FIRST_MACROSACCADE/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     42
arcdeep-4-3-0 :     43
line-8-1-0 :     42
zigzagSq-1-1-1 :     46
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3-0

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=102
level line-8-1-0, keeping, n=89
level arcdeep-4-3-0, keeping, n=136
level zigzagSq-1-1-1, keeping, n=135
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 102', 'arcdeep-4-3-0 : 136', 'line-8-1-0 : 89', 'zigzagSq-1-1-1 : 135']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_FIRST_MACROSACCADE/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     102
arcdeep-4-3-0 :     136
line-8-1-0 :     89
zigzagSq-1-1-1 :     135
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3-0 : 43', 'line-8-1-0 : 42', 'zigzagSq-1-1-1 

M1_l (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3-0 : 43', 'line-8-1-0 : 42', 'zigzagSq-1-1-1 : 46']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_FIRST_MACROSACCADE/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     42
arcdeep-4-3-0 :     43
line-8-1-0 :     42
zigzagSq-1-1-1 :     46
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3-0

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=102
level line-8-1-0, keeping, n=89
level arcdeep-4-3-0, keeping, n=136
level zigzagSq-1-1-1, keeping, n=135
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 102', 'arcdeep-4-3-0 : 136', 'line-8-1-0 : 89', 'zigzagSq-1-1-1 : 135']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_FIRST_MACROSACCADE/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     102
arcdeep-4-3-0 :     136
line-8-1-0 :     89
zigzagSq-1-1-1 :     135
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3-0 : 43', 'line-8-1-0 : 42', 'zigzagSq-1-1-1 

PMv_l (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3-0 : 43', 'line-8-1-0 : 42', 'zigzagSq-1-1-1 : 46']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_FIRST_MACROSACCADE/PMv_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     42
arcdeep-4-3-0 :     43
line-8-1-0 :     42
zigzagSq-1-1-1 :     46
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3

...

preSMA_p (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3-0 : 43', 'line-8-1-0 : 42', 'zigzagSq-1-1-1 : 46']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_FIRST_MACROSACCADE/preSMA_p-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     42
arcdeep-4-3-0 :     43
line-8-1-0 :     42
zigzagSq-1-1-1 :     46
['var_decode', 'V-2-2-0 : 42', 'arcde

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=102
level line-8-1-0, keeping, n=89
level arcdeep-4-3-0, keeping, n=136
level zigzagSq-1-1-1, keeping, n=135
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 102', 'arcdeep-4-3-0 : 136', 'line-8-1-0 : 89', 'zigzagSq-1-1-1 : 135']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_FIRST_MACROSACCADE/preSMA_p-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     102
arcdeep-4-3-0 :     136
line-8-1-0 :     89
zigzagSq-1-1-1 :     135
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3-0 : 43', 'line-8-1-0 : 42', 'zigzagSq-1-

preSMA_a (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3-0 : 43', 'line-8-1-0 : 42', 'zigzagSq-1-1-1 : 46']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_FIRST_MACROSACCADE/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     42
arcdeep-4-3-0 :     43
line-8-1-0 :     42
zigzagSq-1-1-1 :     46
['var_decode', 'V-2-2-0 : 42', 'arcde

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=102
level line-8-1-0, keeping, n=89
level arcdeep-4-3-0, keeping, n=136
level zigzagSq-1-1-1, keeping, n=135
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level zigzagSq-1-1-1, keeping, n=46
level line-8-1-0, keeping, n=42
level V-2-2-0, keeping, n=42
level arcdeep-4-3-0, keeping, n=43
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 102', 'arcdeep-4-3-0 : 136', 'line-8-1-0 : 89', 'zigzagSq-1-1-1 : 135']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_FIRST_MACROSACCADE/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     102
arcdeep-4-3-0 :     136
line-8-1-0 :     89
zigzagSq-1-1-1 :     135
['var_decode', 'V-2-2-0 : 42', 'arcdeep-4-3-0 : 43', 'line-8-1-0 : 42', 'zigzagSq-1-

91125
Plotting bregion:  M1_m
Plotting bregion:  M1_l
Plotting bregion:  PMv_l
Plotting bregion:  PMv_m
Plotting bregion:  PMd_p
Plotting bregion:  PMd_a
Plotting bregion:  dlPFC_a
Plotting bregion:  vlPFC_p
Plotting bregion:  vlPFC_a
Plotting bregion:  FP_p
Plotting bregion:  FP_a
Plotting bregion:  SMA_p
Plotting bregion:  SMA_a
Plotting bregion:  preSMA_p
Plotting bregion:  preSMA_a
=== PLOTTING (all the following plots): prims_single prims_single fixon_preparation fixon_preparation
Plotting var_decode_train, var_decode_test:  seqc_0_shape seqc_0_shape
Plotting var_decode_train, var_decode_test:  seqc_0_shape shape-fixation
Plotting var_decode_train, var_decode_test:  shape-fixation seqc_0_shape
! no data...
Plotting var_decode_train, var_decode_test:  shape-fixation shape-fixation
=== PLOTTING (all the following plots): prims_single prims_on_grid fixon_preparation fixon_preparation
Plotting var_decode_train, var_decode_test:  seqc_0_shape seqc_0_shape
Plotting var_decode_train, var

In [10]:
# rest of macrosaccades
DFallpa_ifmf = DFallpa.copy()

nr = DFallpa_ifmf.shape[0]
for i in range(0,nr):
    pa_old = DFallpa_ifmf.iloc[i]['pa']
    pa_new = pa_old.slice_by_labels('trials', 'shape-macrosaccade-index', list(range(2,11)))
    DFallpa_ifmf.iloc[i]['pa'] = pa_new

SAVEDIR_ANALYSIS = f"/home/kgg/Desktop/eyetracking_analyses/DECODERS/{animal}-{date}"
SAVEDIR = f"{SAVEDIR_ANALYSIS}/cross_temporal_diff_var_train_test_ALLBUTFIRST_MACROSACCADE"
os.makedirs(SAVEDIR, exist_ok=True)

DFRES = decodewrapouterloop_categorical_cross_time_cross_var(DFallpa_ifmf,
                                                     list_var_decode_train_test,
                                                         time_bin_size, slide, savedir=SAVEDIR, extract_params=False,
                                                            ignore_same_events=False, which_level="flex")
if len(DFRES)>0:
    decodewrapouterloop_categorical_cross_time_plot(DFRES, SAVEDIR)

M1_m (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 33', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 36', 'zigzagSq-1-1-1 : 51']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_ALLBUTFIRST_MACROSACCADE/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     33
arcdeep-4-3-0 :     52
line-8-1-0 :     36
zigzagSq-1-1-1 :     51
['var_decode', 'V-2-2-0 : 33', 'arcdeep

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=303
level line-8-1-0, keeping, n=264
level arcdeep-4-3-0, keeping, n=443
level zigzagSq-1-1-1, keeping, n=425
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 303', 'arcdeep-4-3-0 : 443', 'line-8-1-0 : 264', 'zigzagSq-1-1-1 : 425']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_ALLBUTFIRST_MACROSACCADE/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     303
arcdeep-4-3-0 :     443
line-8-1-0 :     264
zigzagSq-1-1-1 :     425
['var_decode', 'V-2-2-0 : 33', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 36', 'zigzag

M1_l (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 33', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 36', 'zigzagSq-1-1-1 : 51']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_ALLBUTFIRST_MACROSACCADE/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     33
arcdeep-4-3-0 :     52
line-8-1-0 :     36
zigzagSq-1-1-1 :     51
['var_decode', 'V-2-2-0 : 33', 'arcdeep

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=303
level line-8-1-0, keeping, n=264
level arcdeep-4-3-0, keeping, n=443
level zigzagSq-1-1-1, keeping, n=425
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 303', 'arcdeep-4-3-0 : 443', 'line-8-1-0 : 264', 'zigzagSq-1-1-1 : 425']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_ALLBUTFIRST_MACROSACCADE/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     303
arcdeep-4-3-0 :     443
line-8-1-0 :     264
zigzagSq-1-1-1 :     425
['var_decode', 'V-2-2-0 : 33', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 36', 'zigzag

PMv_l (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 33', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 36', 'zigzagSq-1-1-1 : 51']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_ALLBUTFIRST_MACROSACCADE/PMv_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     33
arcdeep-4-3-0 :     52
line-8-1-0 :     36
zigzagSq-1-1-1 :     51
['var_decode', 'V-2-2-0 : 33', 'arcde

...

preSMA_p (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 33', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 36', 'zigzagSq-1-1-1 : 51']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_ALLBUTFIRST_MACROSACCADE/preSMA_p-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     33
arcdeep-4-3-0 :     52
line-8-1-0 :     36
zigzagSq-1-1-1 :     51
['var_decode', 'V-2-2-0 : 33', 

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=303
level line-8-1-0, keeping, n=264
level arcdeep-4-3-0, keeping, n=443
level zigzagSq-1-1-1, keeping, n=425
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 303', 'arcdeep-4-3-0 : 443', 'line-8-1-0 : 264', 'zigzagSq-1-1-1 : 425']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_ALLBUTFIRST_MACROSACCADE/preSMA_p-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     303
arcdeep-4-3-0 :     443
line-8-1-0 :     264
zigzagSq-1-1-1 :     425
['var_decode', 'V-2-2-0 : 33', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 36', 'zi

preSMA_a (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 33', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 36', 'zigzagSq-1-1-1 : 51']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_ALLBUTFIRST_MACROSACCADE/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     33
arcdeep-4-3-0 :     52
line-8-1-0 :     36
zigzagSq-1-1-1 :     51
['var_decode', 'V-2-2-0 : 33', 

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=303
level line-8-1-0, keeping, n=264
level arcdeep-4-3-0, keeping, n=443
level zigzagSq-1-1-1, keeping, n=425
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level line-8-1-0, keeping, n=36
level V-2-2-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=51
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 303', 'arcdeep-4-3-0 : 443', 'line-8-1-0 : 264', 'zigzagSq-1-1-1 : 425']
Saved to:  /home/kgg/Desktop/eyetracking_analyses/DECODERS/Diego-230630/cross_temporal_diff_var_train_test_ALLBUTFIRST_MACROSACCADE/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     303
arcdeep-4-3-0 :     443
line-8-1-0 :     264
zigzagSq-1-1-1 :     425
['var_decode', 'V-2-2-0 : 33', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 36', 'zi

91125
Plotting bregion:  M1_m
Plotting bregion:  M1_l
Plotting bregion:  PMv_l
Plotting bregion:  PMv_m
Plotting bregion:  PMd_p
Plotting bregion:  PMd_a
Plotting bregion:  dlPFC_a
Plotting bregion:  vlPFC_p
Plotting bregion:  vlPFC_a
Plotting bregion:  FP_p
Plotting bregion:  FP_a
Plotting bregion:  SMA_p
Plotting bregion:  SMA_a
Plotting bregion:  preSMA_p
Plotting bregion:  preSMA_a
=== PLOTTING (all the following plots): prims_single prims_single fixon_preparation fixon_preparation
Plotting var_decode_train, var_decode_test:  seqc_0_shape seqc_0_shape
Plotting var_decode_train, var_decode_test:  seqc_0_shape shape-fixation
Plotting var_decode_train, var_decode_test:  shape-fixation seqc_0_shape
! no data...
Plotting var_decode_train, var_decode_test:  shape-fixation shape-fixation
=== PLOTTING (all the following plots): prims_single prims_on_grid fixon_preparation fixon_preparation
Plotting var_decode_train, var_decode_test:  seqc_0_shape seqc_0_shape
Plotting var_decode_train, var

# archive

In [ ]:
# to reproduce figure
# for each trial, x=pc1, y=pc2

twind_list = [(-0.2, 0), (0, 0.2)] # relative to event
tbin_dur = 0.1 # windowing
tbin_slide = tbin_dur
reshape_method="trials_x_chanstimes"
pca_reduce=True
npcs_keep_force=6
#twind_overall=twind_list[0]

nbr = DFallpa.shape[0]
fig, axes = plt.subplots(nbr, 2, squeeze=False, figsize=(10, 5*nbr))
# then, can try plotting coding axis
test = None
for i in range(0, nbr):    
    pa = DFallpa.iloc[i]['pa']
    br = DFallpa.iloc[i]['bregion']
    #print(br)
    
    for twind in twind_list:
        twind_i = twind_list.index(twind)
        X, PAfinal, _, _, _ = pa.dataextract_state_space_decode_flex(twind_overall=twind, tbin_dur=tbin_dur, tbin_slide=tbin_slide,
                                                              reshape_method=reshape_method, pca_reduce=pca_reduce,
                                                              npcs_keep_force=npcs_keep_force)
        # get the values of the top 2 dimensions and plot
        scatter_x = np.array([list[0] for list in X])
        scatter_y = np.array([list[1] for list in X])
        n = PAfinal.Xlabels['trials']['shape-fixation']

        ax = axes[i,twind_i]
        #ax.scatter(x,y)
        ax.set_xlabel("PC1")
        ax.set_ylabel("PC2")
        ax.set_title(br+str(twind))
        ax.set_box_aspect(1)
        for g in np.unique(n):
            inds = np.where(n == g)
            ax.scatter(scatter_x[inds], scatter_y[inds], label=g)
plt.show()